In [1]:
#  Enable HTML/CSS 
from IPython.core.display import HTML
HTML("<link href='https://fonts.googleapis.com/css?family=Passion+One' rel='stylesheet' type='text/css'><style>div.attn { font-family: 'Helvetica Neue'; font-size: 30px; line-height: 40px; color: #FFFFFF; text-align: center; margin: 30px 0; border-width: 10px 0; border-style: solid; border-color: #5AAAAA; padding: 30px 0; background-color: #DDDDFF; }hr { border: 0; background-color: #ffffff; border-top: 1px solid black; }hr.major { border-top: 10px solid #5AAA5A; }hr.minor { border: none; background-color: #ffffff; border-top: 5px dotted #CC3333; }div.bubble { width: 65%; padding: 20px; background: #DDDDDD; border-radius: 15px; margin: 0 auto; font-style: italic; color: #f00; }em { color: #AAA; }div.c1{visibility:hidden;margin:0;height:0;}div.note{color:red;}</style>")

___
Enter Team Member Names here (double click to edit):

- Harry Bhasin
- Oscar Padilla
- Najeeb Syed

________

# In Class Assignment Three
In the following assignment you will be asked to fill in python code and derivations for a number of different problems. Please read all instructions carefully and turn in the rendered notebook (or HTML of the rendered notebook)  before the end of class.

<a id="top"></a>
## Contents
* <a href="#Loading">Loading the Data</a>
* <a href="#distance">Measuring Distances</a>

** Available after class begins: **
* <a href="#KNN">K-Nearest Neighbors</a>
* <a href="#naive">Naive Bayes</a>

________________________________________________________________________________________________________
<a id="Loading"></a>
<a href="#top">Back to Top</a>
## Downloading the Document Data
Please run the following code to read in the "20 newsgroups" dataset from sklearn's data loading module.

In [25]:
import logging
logging.basicConfig()

In [26]:
from sklearn.datasets import fetch_20newsgroups_vectorized
import numpy as np
from __future__ import print_function
from scipy.spatial import KDTree
import scipy.sparse as sp

# this takes about 30 seconds to compute, read the next section while this downloads
ds = fetch_20newsgroups_vectorized(subset='train')

# this holds the continuous feature data (which is tfidf)
print('features shape:', ds.data.shape) # there are ~11000 instances and ~130k features per instance
print('target shape:', ds.target.shape) 
print('range of target:', np.min(ds.target),np.max(ds.target))
print('Data type is', type(ds.data), float(ds.data.nnz)/(ds.data.shape[0]*ds.data.shape[1])*100, '% of the data is non-zero')

features shape: (11314, 130107)
target shape: (11314L,)
range of target: 0 19
Data type is <class 'scipy.sparse.csr.csr_matrix'> 0.121435315436 % of the data is non-zero


In [34]:
mtr = sp.csr_matrix(ds.data).toarray()
print('arr: ', mtr)
ds.data
idx = 4000
a = ds.data[idx].todense()

arr:  [[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


In [ ]:
kd = KDTree(mtr, leafsize=10)
kd.query(a, k=2, eps=0, p=2, distance_upper_bound=6)
kd.data

MemoryError: 

## Understanding the Dataset
Look at the description for the 20 newsgroups dataset at http://qwone.com/~jason/20Newsgroups/. You have just downloaded the "vectorized" version of the dataset, which means all the words inside the articles have gone through a transformation that binned them into 130 thousand features related to the words in them.  

**Question Set 1**:
- How many instances are in the dataset? 
- What does each instance represent? 
- How many classes are in the dataset and what does each class represent?
- Would you expect a classifier trained on this data would generalize to documents written in the past week? Why or why not?
- Is the data represented as a sparse or dense matrix?

_______________________
**Answers Question Set 1:**

- 11,314 instances in dataset
- Each instance represents a newsgroup document
- 20 classes, each one representing different newsgroups, i.e. topic
- As described on, http://scikit-learn.org/stable/datasets/twenty_newsgroups.html, "Many classifiers achieve very high F-scores, but their results would not generalize to other documents that aren’t from this window of time." The more the classifier overfits, the smaller the time window where it's going to be effective
- Sparse matrix (Data type is "scipy.sparse.csr.csr_matrix" 0.121435315436 % of the data is non-zero)

___
<a id="distance"></a>
<a href="#top">Back to Top</a>
## Measures of Distance
In the following block of code, we isolate three instances from the dataset. The instance "`a`" is from the group *computer graphics*, "`b`" is from from the group *recreation autos*, and "`c`" is from group *recreation motorcycle*. **Exercise for part 2**: Calculate the: 
- (1) Euclidean distance
- (2) Cosine distance 
- (3) Jaccard similarity 

between each pair of instances using the imported functions below. Remember that the Jaccard similarity is only for binary valued vectors, so convert vectors to binary using a threshold. 

**Question for part 2**: Which distance seems more appropriate to use for this data? **Why**?

In [14]:
from scipy.spatial.distance import cosine
from scipy.spatial.distance import euclidean
from scipy.spatial.distance import jaccard
import numpy as np

# get first instance (comp)
idx = 550
a = ds.data[idx].todense()
a_class = ds.target_names[ds.target[idx]]
print('Instance A is from class', a_class)

# get second instance (autos)
idx = 4000
b = ds.data[idx].todense()
b_class = ds.target_names[ds.target[idx]]
print('Instance B is from class', b_class)

# get third instance (motorcycle)
idx = 7000
c = ds.data[idx].todense()
c_class = ds.target_names[ds.target[idx]]
print('Instance C is from class', c_class)

# Enter distance comparison below for each pair of vectors:

euclidean_ab = euclidean(a, b)
euclidean_ac = euclidean(a, c)
euclidean_bc = euclidean(b, c)

euclidean_measure = (euclidean_ab + euclidean_ac)/2 - euclidean_bc

print('\n\nEuclidean Distance\n ab:', euclidean_ab, 'ac:', euclidean_ac, 'bc:', euclidean_bc, '\n')

cosine_ab = cosine(a, b)
cosine_ac = cosine(a, c)
cosine_bc = cosine(b, c)

cosine_measure = (cosine_ab + cosine_ac)/2 - cosine_bc

print('\nCosine Distance\n ab:', cosine_ab, 'ac:', cosine_ac, 'bc:', cosine_bc)

jaccard_ab = jaccard(a > 0, b > 0)
jaccard_ac = jaccard(a > 0, c > 0)
jaccard_bc = jaccard(b > 0, c > 0)

jaccard_measure = (jaccard_ab + jaccard_ac)/2 - jaccard_bc

print('\nJaccard Dissimilarity (vectors should be boolean values)\n ab:', jaccard_ab, 'ac:', jaccard_ac, 'bc:', jaccard_bc)

print('\n\nThe most appropriate distance is COSINE because it shows the greatest difference between graphics vs. autos&motorcycles', 
      '\nEuclidean: ', euclidean_measure, 
      '\nCosine:    ', cosine_measure, 
      '\nJaccard:  ', jaccard_measure)

Instance A is from class comp.graphics
Instance B is from class rec.autos
Instance C is from class rec.motorcycles


Euclidean Distance
 ab: 1.09851846719 ac: 1.18914054254 bc: 0.917779422666 


Cosine Distance
 ab: 0.603371411376 ac: 0.707027614956 bc: 0.421159534335

Jaccard Dissimilarity (vectors should be boolean values)
 ab: 0.882113821138 ac: 0.875471698113 bc: 0.908794788274


The most appropriate distance is COSINE because it shows the greatest difference between graphics vs. autos&motorcycles 
Euclidean:  0.226050082197 
Cosine:     0.234039978831 
Jaccard:   -0.0300020286479
